# Portfolio Analysis

In [ ]:
!pip install xlrd
!pip install openpyxl
!pip install pyarrow

import datetime
import pyarrow
import yfinance as yf
import pandas as pd

In [ ]:
port = pd.read_csv('sample1.csv', index_col=0)
port.index

In [ ]:
ticker_list = list(port.index)
tickers = yf.Tickers(' '.join(ticker_list))
field_list = ['shortName', 'currency', 'marketCap', 'impliedSharesOutstanding']
stat = pd.DataFrame([{f: tickers.tickers[t].info[f] for f in field_list} for t in ticker_list], index=ticker_list)

In [ ]:
date = datetime.datetime(year=2023, month=12, day=31)

last = pd.Series({t: tickers.tickers[t].history()['Close'].iloc[-1] for t in ticker_list})
stat['last'] = last
stat['marketCapLocal'] = stat['impliedSharesOutstanding'] * stat['last']
stat

In [ ]:
base_currency = 'USD'

# Exclude base currency
d = {base_currency: 1}
for fx in set(stat['currency']) - set([base_currency]):
    # History is limited, say 2003
    hist = yf.Ticker(f'{base_currency}{fx}=X'.upper()).history(period="max")
    rate = hist[hist.index.date <= date.date()].iloc[-1]['Close']
    # Special case for pence
    if fx == 'GBp':
        rate *= 100
    d[fx] = rate
fx_rate = pd.Series(d)
fx_rate

In [ ]:
def history(ticker, end=datetime.datetime.today(), days=30):
    hist = yf.Ticker(ticker).history(period='max').loc[:end.strftime('%Y-%m-%d')]
    return (hist['Close'].iloc[-1], hist['Volume'].iloc[-days:].mean())

# Inclusive of both start/end date; Must provide both
history('NVDA', end=datetime.datetime(year=2023, month=12, day=31))[0]          # 495.22
history('NVDA', end=datetime.datetime(year=2023, month=12, day=31), days=3)[1]  # 28,964,167

In [ ]:
stat['fxRate'] = stat['currency'].map(fx_rate)
stat['marketCapBase'] = stat['marketCapLocal'] / stat['fxRate']
stat['price'] = stat.apply(lambda x: history(x.name, end=date)[0], axis=1)
stat['adtv'] = stat.apply(lambda x: history(x.name, end=date)[1], axis=1)
stat['adtvBase'] = stat['adtv'] * stat['price'] / stat['fxRate']
stat

## Liquidity

In [ ]:
port = pd.read_csv('sample2.csv', index_col=0) # USD 1b

port = pd.read_csv('sample1.csv', index_col=0)
port['value'] = port['weight'] * 1000000000

port.index.equals(stat.index)
analysis = port.merge(stat, left_index=True, right_index=True)

def liquidity(analysis, days=252, participation=0.25):
    if 'shares' in analysis.columns:
        total = (analysis['shares'] * analysis['price'] / analysis['fxRate']).sum()
        liquidate = analysis.apply(lambda x: min(x['shares'], x['adtv'] * participation * days) * x['price'] / x['fxRate'], axis=1).sum()        
    if 'value' in analysis.columns:
        total = analysis['value'].sum()
        liquidate = analysis.apply(lambda x: min(x['value'], x['adtvBase'] * participation * days), axis=1).sum()
    return liquidate / total

days = 10
waterfall = [liquidity(analysis, i, 1/3) for i in range(days)]
pd.Series(waterfall).plot()

In [ ]:
t = yf.Ticker('^GSPC')
px = t.history(period='max')['Close']
px.groupby(pd.Grouper(freq='ME')).last().pct_change().dropna()

# MSCI

MSCI Index Codes and corresponding Bloomberg Tickers

https://www.msci.com/our-solutions/indexes/index-resources/index-tools

Factor performance in different time horizon, by region

https://www.msci.com/factor-index-scorecard/

In [ ]:
def get_last_business_day(d = datetime.datetime.today()):
    return (d - datetime.timedelta(days=max(0, d.weekday() - 4))).date()

# World Factor Indices
codes = [990100, 129896, 136064, 702787, 703755, 705130, 129857, 729749]
# Price = 'STRD', Net = 'NETR', Gross = 'GRTR'
# Currency
# YYYYMMDD, Must be trading date or server error
# Freq = 'DAILY', 'END_OF_MONTH', 'ANNUAL', 
def get_msci(
        codes,
        end_date = get_last_business_day().strftime('%Y%m%d'),
        fx='USD',
        variant='STRD',
        freq='END_OF_MONTH'):
    url = f'https://app2-nv.msci.com/products/service/index/indexmaster/downloadLevelData?output=INDEX_LEVELS&currency_symbol={fx}&index_variant={variant}&start_date=19691231&end_date={end_date}&data_frequency={freq}&baseValue=false&index_codes={",".join(map(str, codes))}'
    return pd.read_excel(url, thousands=',', skiprows=6, skipfooter=19).set_index('Date')


df = get_msci(codes, variant='GRTR').pct_change().dropna()

## Factor

In [ ]:
df2 = df.set_axis(['Index', 'MinVol', 'HiDvdYld', 'Quality', 'Momentum', 'Value', 'Size', 'Growth'], axis=1)
df3 = df2.copy()
df3.iloc[:, 1:] = df2.iloc[:, 1:].sub(df2.iloc[:, 0], axis=0)
df3

y = df2['MinVol']
X = df3

import statsmodels.api as sm
X = sm.add_constant(X)
mod = sm.OLS(y, X)
res = mod.fit()
res.summary()